# Method2 SVD+Huffman

## Import Libraries

In [1]:
import mne
import numpy as np
from scipy.fft import fft,fftshift
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
from scipy.signal import freqz
from scipy import signal
from sklearn.metrics import mean_squared_error
from math import sqrt
import os
import pandas as pd

In [2]:
temp = pd.read_csv('TEMP.csv')
np.savetxt('temp.txt',temp)
temp_array = temp.to_numpy()
temp_array_new = []
for i in temp_array:
    temp_array_new.append(list(i)[0])
N = len(temp_array_new)
sampling_freq = 1/4 
index = np.linspace(0, round((N-1)*sampling_freq,4), N)

## Butterworth Band pass Filter
The butterworth filter were not applied on the TEMP data 
because of it's low frequency and short sequence

## Resampling

In [3]:
y = temp_array
resampled_signal = signal.resample(y,9025)  
#This squared number need to be decided by the users
np.savetxt('processed_temp.txt',resampled_signal)

## SVD

In [4]:
from scipy import linalg
reshaped_signal = np.reshape(resampled_signal, (95,95))
U, s, Vh = linalg.svd(reshaped_signal)

In [5]:
s[2:] = 0

In [6]:
m = 95
n = 95
sigma = np.zeros((m, n))
for i in range(min(m, n)):
    sigma[i, i] = s[i]
reconstructed_signal = np.dot(U, np.dot(sigma, Vh))
deSVD = np.reshape(reconstructed_signal,95*95)

## Round the signal and then do Huffman coding

In [7]:
round_signal= np.round(deSVD,2)
np.savetxt('deSVD_temp.txt',round_signal,fmt='%.2f')

### INSTRUCTION ON HOW TO COMPRESS THE DATA BY HUFFMAN CODING


(I used the package "tcmpr 0.2" and "pyhuff 1.1". These two packages provided the same compression result. So here, we just use "tcmpr 0.2")

1. Open your termial or git bash, enter "pip install tcmpr" to install the "tcmpr 0.2" package
2. Enter the directory which include the file you want to compress OR copy the path of the file you want to compress
3. Enter "tcmpr filename.txt" / "tcmpr filepath" to compress the file
4. Find the compressed file in the same directory of the original file

In [8]:
os.system('tcmpr deSVD_temp.txt')

0

You could find a file name "deSVD_temp.txt.huffman" in the current directory

### After this step, you could calculate the compression ratio if you want

## Decode the data

In [9]:
os.system('tcmpr -d deSVD_temp.txt.huffman')
decoded_data = np.loadtxt(fname = "deSVD_temp.txt")
decoded_data = decoded_data[..., np.newaxis]

In [10]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy import signal
# original_signal = normalized_signal
# compressed_signal = decoded_data
def PRD_calculation(original_signal, compressed_signal):
    PRD = sqrt(sum((original_signal-compressed_signal)**2)/(sum(original_signal**2)))
    return PRD

PRD = PRD_calculation(resampled_signal, decoded_data)
print("The PRD is {}%".format(round(PRD*100,3)))

The PRD is 0.669%


CR = 227/21 = 10.8